In [2]:
import os
import json
import pandas as pd
import numpy as np

In [3]:
input_dir = "raw_oxide_data/"
output_dir = "labels/"

# Load all CIF files in
file_type = ".json"
files = os.listdir(input_dir)
json_files = [file for file in files if file.endswith(file_type)]
# Make Output Directory if needed
if not os.path.isdir(output_dir):
    os.mkdir(output_dir)

labels = {}
for file in json_files:
    f = open(input_dir+file)
    data = json.load(f)
    name = file.split(".json")[0]
    labels[name] = {"structure_name": name}
    labels[name].update({"thermal_expansion": 1e6*data["sample"]["measurement"][0]["property"]["scalar"]})  # IN 1e-6/K UNITS
    if "scalar" in data["sample"]["measurement"][0]["condition"][0].keys():
        labels[name].update({"temperature": data["sample"]["measurement"][0]["condition"][0]["scalar"]})
    else:
        labels[name].update({"temperature": np.mean(data["sample"]["measurement"][0]["condition"][0]["range"])})
        # TODO: dont just use mean of data range
    f.close()
    
# Save to file
labels_df = pd.DataFrame.from_dict(labels).T
labels_df.to_csv(output_dir+"labels.csv")